colab specific path setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/cnn_with_image_example')

In [3]:
os.listdir()

['mri', 'cnn2.ipynb']

In [4]:
for (root,dirs,files) in os.walk('mri'):
    print (root)
    print (dirs)
    print (files)
    print ('--------------------------------')

mri
['yes', 'no']
[]
--------------------------------
mri/yes
[]
['Y73.jpg', 'Y62.jpg', 'Y61.jpg', 'Y60.jpg', 'Y59.JPG', 'Y6.jpg', 'Y58.JPG', 'Y56.jpg', 'Y54.jpg', 'Y55.jpg', 'Y53.jpg', 'Y51.jpg', 'Y52.jpg', 'Y50.JPG', 'Y45.JPG', 'Y46.jpg', 'Y44.JPG', 'Y42.jpg', 'Y4.jpg', 'Y40.JPG', 'Y41.jpg', 'Y97.JPG', 'Y95.jpg', 'Y98.JPG', 'Y92.png', 'Y96.jpg', 'Y91.jpg', 'Y99.JPG', 'Y92.jpg', 'Y90.jpg', 'Y9.jpg', 'Y89.JPG', 'Y86.JPG', 'Y82.jpg', 'Y85.JPG', 'Y79.jpg', 'Y78.jpg', 'Y81.jpg', 'Y76.jpg', 'Y8.jpg', 'Y77.jpg', 'Y75.JPG', 'Y71.JPG', 'Y7.jpg', 'Y70.jpg', 'Y74.jpg', 'Y69.jpg', 'Y66.JPG', 'Y65.JPG', 'Y67.JPG', 'Y112.JPG', 'Y103.jpg', 'Y107.jpg', 'Y117.JPG', 'Y167.JPG', 'Y169.jpg', 'Y17.jpg', 'Y20.jpg', 'Y21.jpg', 'Y185.jpg', 'Y15.jpg', 'Y168.jpg', 'Y22.jpg', 'Y18.JPG', 'Y23.JPG', 'Y100.JPG', 'Y148.JPG', 'Y165.JPG', 'Y108.jpg', 'Y170.JPG', 'Y180.jpg', 'Y16.JPG', 'Y24.jpg', 'Y166.JPG', 'Y184.JPG', 'Y146.JPG', 'Y194.jpg', 'Y183.jpg', 'Y109.JPG', 'Y113.JPG', 'Y155.JPG', 'Y162.jpg', 'Y13.jpg', 'Y1

main code starts now

In [5]:
import tensorflow as tf

In [6]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [7]:
train_data = train_gen.flow_from_directory('mri',target_size=(64,64),color_mode='grayscale',class_mode='binary')

Found 253 images belonging to 2 classes.


In [8]:
train_data.class_indices

{'no': 0, 'yes': 1}

In [9]:
type(train_gen)

keras.preprocessing.image.ImageDataGenerator

nn architecture

In [18]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation=tf.keras.activations.relu,input_shape=(64,64,1)))
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(2,activation=tf.keras.activations.softmax))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 16)        160       
                                                                 
 dropout (Dropout)           (None, 62, 62, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 16)        2320      
                                                                 
 dropout_1 (Dropout)         (None, 29, 29, 16)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                  

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [28]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/logs", histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)

In [30]:
%tensorboard --logdir /content/logs

Reusing TensorBoard on port 6007 (pid 1255), started 0:25:32 ago. (Use '!kill 1255' to kill it.)

<IPython.core.display.Javascript object>

In [23]:
history = model.fit_generator(train_data, epochs=5, 
                              callbacks=[early_stopping, tensorboard_callback],
                              use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/5
8/8 [==============================] - 2s 185ms/step - loss: 0.6116 - accuracy: 0.6601
Epoch 2/5
8/8 [==============================] - 2s 275ms/step - loss: 0.5116 - accuracy: 0.7708
Epoch 3/5
8/8 [==============================] - 2s 254ms/step - loss: 0.4679 - accuracy: 0.7866
Epoch 4/5
8/8 [==============================] - 2s 238ms/step - loss: 0.4236 - accuracy: 0.8142
Epoch 5/5
8/8 [==============================] - 2s 262ms/step - loss: 0.4039 - accuracy: 0.8300
